import csv
import redis

r = redis.Redis(host="localhost", port=6379, decode_responses=True)

with open("clean_file_bueno.csv", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    
    for row in reader:
        nombre = row["names"]

        r.hset(f"pelicula:{nombre}", mapping={
            "nombre": nombre,
            "fecha": row["date_x"],
            "score": row["score"],
            "genero": row["genre"]
        })

Ver un ejemplo
print(r.hgetall("pelicula:Avatar: The Way of Water"))

In [27]:
import csv
import redis

r = redis.Redis(host='redis-server', port=6379, db=0, decode_responses=True)

### **Para cada película crea un HASH con clave**

In [23]:
with open("./clean_file_bueno.csv", encoding="utf-8") as f:
    reader = csv.DictReader(f)

    for row in reader:
        nombre = row["names"]
        fecha = row["date_x"]
        score = float(row["score"])
        genero = row["genre"]

        r.hset(f"pelicula:{nombre}", mapping={
            "nombre": nombre,
            "fecha": fecha,
            "score": score,
            "genero": genero
        })



### **Crear un ranking con puntuaciones (ZSET)**

In [24]:
with open("./clean_file_bueno.csv", encoding="utf-8") as f:
    reader = csv.DictReader(f)

    for row in reader:
        nombre = row["names"]
        score = float(row["score"])

        r.zadd("peliculas:ranking", {nombre: score})

### **De una peli sacar**

In [25]:
with open("./clean_file_bueno.csv", encoding="utf-8") as f:
    reader = csv.DictReader(f)

    for row in reader:
        nombre = row["names"]
        fecha = row["date_x"]
        score = float(row["score"])
        genero = row["genre"]
        pais = row["country"]

        # HASH
        r.hset(f"pelicula:{nombre}", mapping={
            "nombre": nombre,
            "fecha": fecha,
            "score": score,
            "genero": genero,
            "pais": pais
        })

        # ZSET ranking
        r.zadd("peliculas:ranking", {nombre: score})

        # SET por país
        r.sadd(f"pais:{pais}", nombre)

        # LISTAS por género
        generos_lista = genero.split("\t")
        for g in generos_lista:
            r.rpush(f"genero:{g}", nombre)

print("Datos cargados correctamente.")

Datos cargados correctamente.


In [26]:
# **Cuando necesitas procesar una lista hasta vaciarla**
while r.llen("pedidos") > 0:
    
    pedido = r.lpop("pedidos")
    
    print("Procesado:", pedido)

In [ ]:
import csv
import redis

# Conectar a Redis
r = redis.Redis(host="redis", port=6379, decode_responses=True)

# --- 1. Cargar CSV y usar solo SADD para agrupar por país ---

with open("./clean_file_bueno.csv", encoding="utf-8") as f:
    reader = csv.DictReader(f)

    for row in reader:
        nombre = row["names"]      # Nombre de la película
        pais = row["country"]      # País (AU, US, MX, etc.)

        # SADD -> Añadir la película al grupo de su país
        r.sadd(f"pais:{pais}", nombre)

print("Películas agrupadas por país con SADD.\n")


# --- 2. Usar SCARD para contar películas por país ---

print("Número de películas en Australia (AU):", r.scard("pais:AU"))
print("Número de películas en Estados Unidos (US):", r.scard("pais:US"))
print("Número de películas en México (MX):", r.scard("pais:MX"))
print()


# --- 3. Usar SMEMBERS para ver todas las películas de un país ---

print("Películas de Australia (AU):")
print(r.smembers("pais:AU"))
print()

print("Películas de Estados Unidos (US):")
print(r.smembers("pais:US"))


### **Impresion**

**SETS — SADD, SMEMBERS, SCARD**

**Imprimir todos los elementos del set:**

print("Películas de AU:")
print(r.smembers("pais:AU"))

**Imprimir cuántos elementos hay:**

print("Número de películas:", r.scard("pais:AU"))

**LISTAS — LRANGE**

tareas = r.lrange("tareas", 0, -1)

print("Tareas actuales:", tareas)

**Imprimir elemento por elemento:**

for t in r.lrange("tareas", 0, -1):
    print("Tarea:", t)

**HASHES — HGET, HGETALL**

**Imprimir un campo:**

print("Score:", r.hget("pelicula:Avatar", "score"))

**Imprimir todo el hash:**

info = r.hgetall("pelicula:Avatar")

print("Información de Avatar:")

print(info)

**Imprimir bonito:**

for campo, valor in r.hgetall("pelicula:Avatar").items():

    print(campo, ":", valor)

**ZSET — ZREVRANGE**

**Imprimir el top 5:**

top5 = r.zrevrange("ranking", 0, 4, withscores=True)

print("TOP 5 películas:")

print(top5)

**Imprimir formato más bonito:**

for i, (pelicula, score) in enumerate(top5, start=1):

    print(f"{i}. {pelicula} → {score}")

**GEO — GEOSEARCH**

puntos = r.geosearch("poi", longitude=-3.70, latitude=40.41, radius=2, unit="km")

print("Resultados de búsqueda GEOSearch:")

print(puntos)

**Ejemplo**

top10 = r.zrevrange("peliculas:ranking", 0, 9, withscores=True)

print("TOP 10 PELÍCULAS:")

for i, (nombre, score) in enumerate(top10, start=1):

    print(f"{i}. {nombre} → {score}")

In [ ]:
**Ubicaciones**
import csv
import redis

# ------------------------------
# CONEXIÓN A REDIS
# ------------------------------
r = redis.Redis(host="redis", port=6379, decode_responses=True)

# ------------------------------
# 1. CARGAR POIs DESDE CSV
# ------------------------------

with open("pois.csv", encoding="utf-8") as f:
    reader = csv.DictReader(f)

    for row in reader:
        poi_id = row["id"]
        nombre = row["name"]
        lon = float(row["lon"])
        lat = float(row["lat"])

        # Insertar coordenadas en GEOADD
        r.geoadd("poi:locations", (lon, lat, poi_id))

        # Guardar nombre del POI
        r.hset("poi:info", poi_id, nombre)

print(">>> Todos los POIs han sido cargados correctamente.\n")


# ------------------------------
# 2. GEOSEARCH: BUSCAR POIs CERCA DEL USUARIO
# ------------------------------

# Ubicación del usuario (Ejemplo: Puerta del Sol)
lon_u = -3.703790
lat_u = 40.416775

# Buscar POIs en un radio de 2 km
ids = r.geosearch(
    "poi:locations",
    longitude=lon_u,
    latitude=lat_u,
    radius=2000,   # 2 km
    unit="m"
)

print(f"--- POIs encontrados en 2 km: {len(ids)} ---")

for poi_id in ids:
    nombre = r.hget("poi:info", poi_id)
    print(f"→ {nombre} ({poi_id})")

print()


# ------------------------------
# 3. DISTANCIA EXACTA (Opcional)
# ------------------------------
print("--- Distancias desde el usuario ---")
for poi_id in ids:
    distancia = r.geodist("poi:locations", poi_id, ids[0], "km") if len(ids)>1 else None
    nombre = r.hget("poi:info", poi_id)
    print(f"{nombre}: {distancia} km (comparado con primer POI encontrado)")

print()


# ------------------------------
# 4. BUSCAR EL POI MÁS CERCANO
# ------------------------------

nearest = r.geosearch(
    "poi:locations",
    longitude=lon_u,
    latitude=lat_u,
    radius=50000,   # busca en gran radio
    unit="m",
    sort="ASC",     # primero el más cercano
    count=1         # solo uno
)

if nearest:
    poi_id = nearest[0]
    nombre = r.hget("poi:info", poi_id)

    distancia = r.geodist("poi:locations", poi_id, poi_id, "km")

    print(">>> POI MÁS CERCANO:")
    print(f"{nombre} ({poi_id})")
else:
    print("No hay POIs cerca del usuario.")

In [30]:
r.flushdb()

True

In [ ]:
import redis
r = redis.Redis(host="localhost", port=6379, decode_responses=True)

# ============================
# 1) LISTAS (SIN CSV)
# ============================
r.delete("tareas")

r.rpush("tareas", "hacer la compra")
r.rpush("tareas", "estudiar redis")
r.lpush("tareas", "URGENTE: pagar agua")

print("LISTA COMPLETA:", r.lrange("tareas", 0, -1))

while r.llen("tareas") > 0:
    print("Procesando:", r.lpop("tareas"))


# ============================
# 2) HASHES (SIN CSV)
# ============================
r.delete("usuario:1")

r.hset("usuario:1", mapping={
    "nombre": "Ana",
    "pais": "ES",
    "edad": "23"
})

print("HASH COMPLETO:", r.hgetall("usuario:1"))
print("Nombre:", r.hget("usuario:1", "nombre"))


# ============================
# 3) SETS (SIN CSV)
# ============================
r.delete("paises")

r.sadd("paises", "ES")
r.sadd("paises", "MX")
r.sadd("paises", "CL")

print("Miembros:", r.smembers("paises"))
print("Total:", r.scard("paises"))


# ============================
# 4) SORTED SET (RANKING)
# ============================
r.delete("ranking")

r.zadd("ranking", {"Ana": 300})
r.zadd("ranking", {"Luis": 500})
r.zadd("ranking", {"Maria": 200})

print("TOP 2:", r.zrevrange("ranking", 0, 1, withscores=True))


# ============================
# 5) HYPERLOGLOG
# ============================
r.delete("logins")

r.pfadd("logins", "ana")
r.pfadd("logins", "luis")
r.pfadd("logins", "ana")   # no duplica

print("Usuarios únicos:", r.pfcount("logins"))


# ============================
# 6) GEO
# ============================
r.delete("poi")

r.geoadd("poi", (-3.7038, 40.4168, "sol"))    # Puerta del Sol
r.geoadd("poi", (-3.6921, 40.4138, "prado"))  # Museo del Prado

result = r.geosearch(
    "poi",
    longitude=-3.70,
    latitude=40.41,
    radius=3,
    unit="km"
)

print("POIs cercanos:", result)


In [ ]:
import redis
import csv

r = redis.Redis(host="localhost", port=6379, decode_responses=True)

# Limpieza previa
r.flushdb()

with open("clean_file_bueno.csv", encoding="utf-8") as f:
    reader = csv.DictReader(f)

    for row in reader:
        nombre = row["names"]
        fecha = row["date_x"]
        score = float(row["score"])
        genero = row["genre"]
        pais = row["country"]

        # ============================
        # 1) GUARDAR CADA PELÍCULA EN UN HASH
        # ============================
        r.hset(f"pelicula:{nombre}", mapping={
            "nombre": nombre,
            "fecha": fecha,
            "score": score,
            "genero": genero,
            "pais": pais
        })

        # ============================
        # 2) CREAR UN RANKING
        # ============================
        r.zadd("peliculas:ranking", {nombre: score})

        # ============================
        # 3) CLASIFICAR POR PAÍS (SET)
        # ============================
        r.sadd(f"pais:{pais}", nombre)

        # ============================
        # 4) CLASIFICAR POR GÉNERO (LISTA)
        # ============================
        generos = genero.split("\t")   # el CSV tiene géneros separados por tabulador
        for g in generos:
            r.rpush(f"genero:{g}", nombre)


print("✔ CSV cargado en Redis correctamente")


In [ ]:
Mostrar una película
print(r.hgetall("pelicula:Avatar: The Way of Water"))

In [ ]:
top10 = r.zrevrange("peliculas:ranking", 0, 9, withscores=True)
for i, (nombre, score) in enumerate(top10, start=1):
    print(i, nombre, score)


In [ ]:
Películas por país
print(r.smembers("pais:AU"))

In [ ]:
Películas por género
print(r.lrange("genero:Action", 0, -1))

In [ ]:
Listar TODAS las películas guardadas
for key in r.keys("pelicula:*"):
    print(r.hgetall(key))

In [ ]:
Contar cuántas películas del CSV pertenecen a cada país
for pais_key in r.keys("pais:*"):
    print(pais_key, "→", r.scard(pais_key))

In [ ]:
import csv
import redis
import json

r = redis.Redis(host="localhost", port=6379, decode_responses=True)

with open("clean_file_bueno.csv", encoding="utf-8") as f:
    reader = csv.DictReader(f)

    for row in reader:
        pelicula = {
            "nombre": row["names"],
            "fecha": row["date_x"],
            "genero": row["genre"]
        }
        
        pelicula_json = json.dumps(pelicula)

        # Añadir al final de la lista
        r.rpush("lista:peliculas", pelicula_json)

print("Películas agregadas a la lista.")

def procesar_pelicula():
    pelicula_json = r.lpop("lista:peliculas")
    if pelicula_json:
        pelicula = json.loads(pelicula_json)
        print("Procesando película:", pelicula)
    else:
        print("No hay más películas.")

lista = r.lrange("lista:peliculas", 0, -1)
print("\nPelículas actuales en cola:\n")
for item in lista:
    print(json.loads(item))

def agregar_pelicula_urgente(nombre, fecha, genero):
    pelicula = {"nombre": nombre, "fecha": fecha, "genero": genero}
    pelicula_json = json.dumps(pelicula)

    r.lpush("lista:peliculas", pelicula_json)
    print("PELÍCULA URGENTE AGREGADA:", pelicula)


In [ ]:
with open("clean_file_bueno.csv", encoding="utf-8") as f:
    reader = csv.DictReader(f)

    for row in reader:
        r.rpush("lista_peliculas", row["names"])

print(r.lrange("lista_peliculas", 0, -1))

primera = r.lpop("lista_peliculas")
print("Procesando:", primera)

with open("clean_file_bueno.csv", encoding="utf-8") as f:
    reader = csv.DictReader(f)

    for row in reader:
        score = float(row["score"])
        if score < 60:
            r.rpush("peliculas_malas", row["names"])



In [ ]:
import csv
import redis

r = redis.Redis(host="localhost", port=6379, decode_responses=True)

with open("clean_file_bueno.csv", encoding="utf-8") as f:
    reader = csv.DictReader(f)

    for row in reader:
        pais = row["country"]

        # contador por país
        r.incr(f"contador:{pais}")

        # guardar última película insertada
        r.set("ultima_pelicula", row["names"])

print("Última película:", r.get("ultima_pelicula"))
print("Películas de AU:", r.get("contador:AU"))
